Based on the definitions of pseudolikelihood for Ising model parameters in
H. Chau Nguyen, Riccardo Zecchina & Johannes Berg (2017)
Inverse statistical problems: from the inverse Ising problem to data science,
Advances in Physics, 66:3, 197-261, DOI: 10.1080/00018732.2017.1341604
and
Aurell, E., & Ekeberg, M. (2012).
Inverse Ising inference using all the data.
Physical review letters, 108(9), 090201.

In [1]:
import os
import torch
import time

In [2]:
data_directory = 'E:\\HCP_data'
output_directory = 'E:\\Ising_model_results_batch'
learning_rate = 0.001
threshold = 0.1# Threshold for binarizing time series data after z-scoring.
beta = 0.5# Factor we can use to adjust the chance of accepting a flip that increases the energy.
num_nodes = 360
num_epochs = 100
num_reps = 5
data_subset = 'training'
subjects_start = 0
subjects_end = 10# 699
code_start_time = time.time()
int_type = torch.int
float_type = torch.float
device = torch.device('cuda')

In [3]:
def get_fc(ts:torch.Tensor):
    # We want to take the correlations between pairs of areas over time.
    # torch.corrcoef() assumes observations are in columns and variables in rows.
    # As such, we have to take the transpose.
    return torch.corrcoef( ts.transpose(dim0=0, dim1=1) )

def get_fc_batch(ts_batch:torch.Tensor):
    # Take the FC of each individual time series.
    batch_size = ts_batch.size(dim=0)
    num_rois = ts_batch.size(dim=-1)
    fc = torch.zeros( (batch_size, num_rois, num_rois), dtype=ts_batch.dtype, device=ts_batch.device )
    for ts_index in range(batch_size):
        fc[ts_index,:,:] = get_fc(ts_batch[ts_index,:,:])
    return fc

def get_fcd(ts:torch.Tensor, window_length:int=90, window_step:int=1):
    # Calculate functional connectivity dynamics.
    # Based on Melozzi, Francesca, et al. "Individual structural features constrain the mouse functional connectome." Proceedings of the National Academy of Sciences 116.52 (2019): 26961-26969.
    dtype = ts.dtype
    device = ts.device
    ts_size = ts.size()
    T = ts_size[0]
    R = ts_size[1]
    triu_index = torch.triu_indices(row=R, col=R, offset=1, dtype=torch.int, device=device)
    triu_row = triu_index[0]
    triu_col = triu_index[1]
    num_indices = triu_index.size(dim=1)
    left_window_margin = window_length//2 + 1
    right_window_margin = window_length - left_window_margin
    window_centers = torch.arange(start=left_window_margin, end=T-left_window_margin+1, step=window_step, dtype=torch.int, device=device)
    window_offsets = torch.arange(start=-right_window_margin, end=left_window_margin, dtype=torch.int, device=device)
    num_windows = window_centers.size(dim=0)
    window_fc = torch.zeros( (num_windows, num_indices), dtype=dtype, device=device )
    for c in range(num_windows):
        fc = get_fc(ts[window_offsets+window_centers[c],:])
        window_fc[c,:] = fc[triu_row, triu_col]
    return torch.corrcoef(window_fc)

# only assumes the tensors are the same shape
def get_rmse(tensor1:torch.Tensor, tensor2:torch.Tensor):
    return torch.sqrt(  torch.mean( torch.square(tensor2 - tensor1) )  )

# takes the mean over all but the first dimension
# For our 3D batch-first tensors, this gives us a 1D tensor of RMSE values, one for each batch.
def get_rmse_batch(tensor1:torch.Tensor, tensor2:torch.Tensor):
    dim_indices = tuple(   range(  len( tensor1.size() )  )   )
    return torch.sqrt(  torch.mean( torch.square(tensor2 - tensor1), dim=dim_indices[1:] )  )

# In several cases, we have symmetric square matrices with fixed values on the diagonal.
# In particular, this is true of the functional connectivity and structural connectivity matrices.
# For such matrices, it is more meaningful to only calculate the RMSE of the elements above the diagonal.
def get_triu_rmse(tensor1:torch.Tensor, tensor2:torch.Tensor):
    indices = torch.triu_indices( row=tensor1.size(0), col=tensor1.size(1), offset=1, device=tensor1.device )
    indices_r = indices[0]
    indices_c = indices[1]
    return get_rmse( tensor1[indices_r,indices_c], tensor2[indices_r,indices_c] )

def get_triu_rmse_batch(tensor1:torch.Tensor, tensor2:torch.Tensor):
    indices = torch.triu_indices( tensor1.size(-2), tensor1.size(-1), 1 )
    indices_r = indices[0]
    indices_c = indices[1]
    batch_size = tensor1.size(dim=0)
    num_triu_elements = indices.size(dim=1)
    dtype = tensor1.dtype
    device = tensor1.device
    triu1 = torch.zeros( (batch_size, num_triu_elements), dtype=dtype, device=device )
    triu2 = torch.zeros_like(triu1)
    for b in range(batch_size):
        triu1[b,:] = tensor1[b,indices_r,indices_c]
        triu2[b,:] = tensor2[b,indices_r,indices_c]
    return get_rmse_batch(triu1, triu2)

def get_triu_corr(tensor1:torch.Tensor, tensor2:torch.Tensor):
    indices = torch.triu_indices( row=tensor1.size(0), col=tensor1.size(1), offset=1, device=tensor1.device )
    indices_r = indices[0]
    indices_c = indices[1]
    tensor1_triu = tensor1[indices_r,indices_c].unsqueeze(dim=0)
    # print( tensor1_triu.size() )
    tensor2_triu = tensor2[indices_r,indices_c].unsqueeze(dim=0)
    # print( tensor2_triu.size() )
    tensor_pair_triu = torch.cat( (tensor1_triu,tensor2_triu), dim=0 )
    # print( tensor_pair_triu.size() )
    corr = torch.corrcoef(tensor_pair_triu)
    # print(corr)
    return corr[0,1]

def get_triu_corr_batch(tensor1:torch.Tensor, tensor2:torch.Tensor):
    indices = torch.triu_indices( tensor1.size(-2), tensor1.size(-1), 1 )
    indices_r = indices[0]
    indices_c = indices[1]
    batch_size = tensor1.size(dim=0)
    dtype = tensor1.dtype
    device = tensor1.device
    corr_batch = torch.zeros( (batch_size,), dtype=dtype, device=device )
    for b in range(batch_size):
        triu_pair = torch.stack( (tensor1[b,indices_r,indices_c], tensor2[b,indices_r,indices_c]) )
        pair_corr = torch.corrcoef(triu_pair)
        corr_batch[b] = pair_corr[0,1]
    return corr_batch

In [4]:
def get_data_time_series_from_single_file(data_directory:str, data_subset:str, subjects_start:int, subjects_end:int, num_nodes:int, data_threshold:float):
    ts_file = os.path.join( data_directory, f'sc_fmri_ts_{data_subset}.pt' )
    data_ts = torch.load(ts_file)[subjects_start:subjects_end,:,:,:num_nodes]
    data_std, data_mean = torch.std_mean(data_ts, dim=-2, keepdim=True)
    data_ts -= data_mean
    data_ts /= data_std
    data_ts = data_ts.flatten(start_dim=1, end_dim=-2)
    data_ts = torch.sign(data_ts - data_threshold)
    return data_ts
    
def get_batched_ising_models(batch_size:int, num_nodes:int):
    J = torch.randn( (batch_size, num_nodes, num_nodes), dtype=float_type, device=device )
    J = 0.5*( J + J.transpose(dim0=-2, dim1=-1) )
    J = J - torch.diag_embed( torch.diagonal(J,dim1=-2,dim2=-1) )
    h = torch.zeros( (batch_size, num_nodes), dtype=float_type, device=device )
    s = 2.0*torch.randint( 2, (batch_size, num_nodes), dtype=float_type, device=device ) - 1.0
    return J, h, s
    
def print_if_nan(mat:torch.Tensor):
    mat_is_nan = torch.isnan(mat)
    num_nan = torch.count_nonzero(mat_is_nan)
    if num_nan > 0:
        print( mat.size(), f'matrix contains {num_nan} NaNs.' )

In [5]:
# Load, normalize, binarize, and flatten the fMRI time series data.
print('loading fMRI data...')
# training_subjects = load_training_subjects(directory_path=data_directory)
# data_fc, data_cov, data_mu = get_data_time_series_stats(data_directory=data_directory, subjects=training_subjects, time_series_per_subject=time_series_per_subject, steps_per_time_series=num_time_points, num_nodes=num_nodes, data_threshold=threshold, window_length=num_steps)
data_ts = get_data_time_series_from_single_file(data_directory=data_directory, data_subset=data_subset, subjects_start=subjects_start, subjects_end=subjects_end, num_nodes=num_nodes, data_threshold=threshold)
num_subjects, num_time_points, num_nodes = data_ts.size()
print('loaded with subjects x time points x nodes: ', num_subjects, num_time_points, num_nodes)
data_fc = get_fc_batch(data_ts)

loading fMRI data...
loaded with subjects x time points x nodes:  10 4800 360


In [6]:
J, h, s = get_batched_ising_models(batch_size=num_subjects, num_nodes=num_nodes)

In [17]:
def get_negative_log_pseudolikelihood_loss_single_subject(h:torch.Tensor, J:torch.Tensor, data_ts:torch.Tensor, node_index:int, beta:float):
    not_node_indices = torch.arange( data_ts.size(dim=-1), dtype=int_type, device=device ) != node_index
    node_ts = data_ts[:,node_index]
    not_node_ts = data_ts[:,not_node_indices]
    J_not_node_vs_node = J[not_node_indices,node_index]
    h_node = h[node_index]
    ts_J_product = torch.matmul(not_node_ts, J_not_node_vs_node)
    print( not_node_ts.size(), J_not_node_vs_node.size(), ts_J_product.size() )
    deltaH = node_ts*( h_node + ts_J_product )
    nlpll_per_time_point = torch.log( 1+torch.exp(-2*beta*deltaH) )
    nlpll = torch.mean(nlpll_per_time_point)
    return nlpll

In [18]:
single_subject_data_ts = data_ts[0,:,:]
single_subject_J = J[0,:,:]
single_subject_h = h[0,:]
single_subject_s = s[0,:]

In [19]:
single_subject_nlpll = get_negative_log_pseudolikelihood_loss_single_subject(single_subject_h, single_subject_J, single_subject_data_ts, 0, beta)
print(single_subject_nlpll)

torch.Size([4800, 359]) torch.Size([359]) torch.Size([4800])
tensor(3.0654, device='cuda:0')


In [30]:
def get_negative_log_pseudolikelihood_loss(h:torch.Tensor, J:torch.Tensor, data_ts:torch.Tensor, node_index:int, beta:float):
    not_node_indices = torch.arange( data_ts.size(dim=-1), dtype=int_type, device=device ) != node_index
    node_ts = data_ts[:,:,node_index]
    print( node_ts.size() )
    not_node_ts = data_ts[:,:,not_node_indices]
    print( not_node_ts.size() )
    J_not_node_vs_node = J[:,not_node_indices,node_index].unsqueeze(dim=-1)
    print( J_not_node_vs_node.size() )
    h_node = h[:,node_index].unsqueeze(dim=-1)
    print( h_node.size() )
    ts_J_product = torch.matmul(not_node_ts, J_not_node_vs_node).squeeze()
    print( not_node_ts.size(), J_not_node_vs_node.size(), ts_J_product.size() )
    deltaH = node_ts*( h_node + ts_J_product )
    nlpll_per_time_point = torch.log( 1+torch.exp(-2*beta*deltaH) )
    nlpll = torch.mean(nlpll_per_time_point)
    return nlpll

In [31]:
nlpll = get_negative_log_pseudolikelihood_loss(h, J, data_ts, 0, beta)
print(nlpll)

torch.Size([10, 4800])
torch.Size([10, 4800, 359])
torch.Size([10, 359, 1])
torch.Size([10, 1])
torch.Size([10, 4800, 359]) torch.Size([10, 359, 1]) torch.Size([10, 4800])
tensor(5.2127, device='cuda:0')


In [33]:
num_subjects, num_time_points, num_nodes = data_ts.size()
node_index = 0
not_node_indices = torch.arange( data_ts.size(dim=-1), dtype=int_type, device=device ) != node_index
node_data_ts = data_ts[:,:,node_index].reshape( (num_subjects, num_time_points, 1) )
not_node_data_ts = data_ts[:,:,not_node_indices].reshape( (num_subjects, num_time_points, num_nodes-1) )
J_not_node_vs_node = J[:,not_node_indices,node_index].reshape( (num_subjects, num_nodes-1, 1) )
h_node = h[:,node_index].reshape( (num_subjects, 1, 1) )

In [36]:
def get_negative_log_pseudolikelihood_loss_node(h_node:torch.Tensor, J_not_node_vs_node:torch.Tensor, node_data_ts:torch.Tensor, not_node_data_ts:torch.Tensor, beta:float):
    ts_J_product = torch.matmul(not_node_ts, J_not_node_vs_node)
    print( ts_J_product.size() )
    deltaH = node_ts*( h_node + ts_J_product )
    print( deltaH.size() )
    nlpll_per_time_point = torch.log( 1+torch.exp(-2*beta*deltaH) )
    print( nlpll_per_time_point.size() )
    nlpll = torch.mean(nlpll_per_time_point)
    return nlpll

In [37]:
nlpll = get_negative_log_pseudolikelihood_loss_node(h_node, J_not_node_vs_node, node_data_ts, not_node_data_ts, beta)
print(nlpll)

torch.Size([10, 4800, 1])
torch.Size([10, 4800, 1])
torch.Size([10, 4800, 1])
tensor(5.2127, device='cuda:0')


In [6]:
class IsingModelNegativeLogPseudoLikelihoodNode(torch.nn.Module):
    def __init__(self, num_subjects:int, num_other_nodes:int, dtype=float_type, device=device):
        super(IsingModelNegativeLogPseudoLikelihoodNode, self).__init__()
        self.h_node = torch.nn.Parameter( torch.randn( (num_subjects, 1, 1), dtype=dtype, device=device ) )
        self.J_not_node_vs_node = torch.nn.Parameter(  torch.randn( (num_subjects, num_other_nodes, 1), dtype=dtype, device=device )  )
    def forward(self, node_ts:torch.Tensor, not_node_ts):
        deltaH = node_ts * ( self.h_node + torch.matmul(not_node_ts, self.J_not_node_vs_node) )
        return torch.mean(  torch.log( 1+torch.exp(-2*beta*deltaH) )  )

In [13]:
imnlpln_fn = IsingModelNegativeLogPseudoLikelihoodNode(num_subjects=num_subjects, num_other_nodes=num_nodes-1, dtype=float_type, device=device)

In [48]:
print( imnlpln_fn(node_ts, not_node_ts) )

tensor(6.0969, device='cuda:0', grad_fn=<MeanBackward0>)


In [14]:
optimizer = torch.optim.Adam( imnlpln_fn.parameters(), lr=learning_rate )

In [52]:
start_time = time.time()
last_time = start_time
print_every_seconds = 10
num_epochs = 100000
batch_size = 30
num_batches = num_time_points//batch_size
for epoch in range(num_epochs):
    time_point_order = torch.randperm(num_time_points, dtype=torch.int, device=device)
    node_data_ts_shuffled = node_data_ts[:,time_point_order,:]
    not_node_data_ts_shuffled = not_node_data_ts[:,time_point_order,:]
    for batch in range(num_batches):
        batch_start = batch*batch_size
        batch_end = batch_start+batch_size
        node_data_ts_batch = node_data_ts_shuffled[:,batch_start:batch_end,:]
        not_node_data_ts_batch = not_node_data_ts_shuffled[:,batch_start:batch_end,:]
        optimizer.zero_grad()
        loss = imnlpln_fn(node_data_ts_batch, not_node_data_ts_batch)
        loss.backward()
        optimizer.step()
        current_time = time.time()
    if current_time - last_time >= print_every_seconds:
        loss = imnlpln_fn(node_ts, not_node_ts)
        print( f'{epoch}, {loss.item()}, {current_time-start_time}' )
        last_time = current_time
loss = imnlpln_fn(node_ts, not_node_ts)
print( f'{epoch}, {loss.item()}, {current_time-start_time}' )

32, 0.09993485361337662, 10.052844762802124
65, 0.0999213457107544, 21.06029486656189
102, 0.09994980692863464, 32.57723259925842
139, 0.0997890830039978, 44.10251569747925
176, 0.09996267408132553, 55.633185386657715
213, 0.09971702098846436, 67.16577410697937
250, 0.09972579777240753, 78.72466254234314
288, 0.09994123876094818, 90.37964630126953
325, 0.09972061961889267, 101.92306971549988
363, 0.10005922615528107, 113.53110265731812
400, 0.09957818686962128, 125.08423233032227
437, 0.09998591989278793, 136.63682317733765
474, 0.10003670305013657, 148.1676332950592
510, 0.09994084388017654, 159.6745047569275
547, 0.10023267567157745, 171.2209768295288
584, 0.09971301257610321, 182.7434310913086
621, 0.0998874381184578, 194.23862147331238
658, 0.09974561631679535, 205.8098955154419
695, 0.09975584596395493, 217.39714908599854
732, 0.09970827400684357, 228.9720995426178
769, 0.09968381375074387, 240.52939891815186
806, 0.09986956417560577, 252.13798189163208
842, 0.09990838170051575, 2

6743, 0.0997781753540039, 2109.627955198288
6779, 0.09985577315092087, 2121.13410615921
6815, 0.0996626690030098, 2132.57283616066
6852, 0.09968791902065277, 2144.142006635666
6889, 0.10004010051488876, 2155.677139520645
6926, 0.09986384212970734, 2167.228801012039
6963, 0.09991485625505447, 2178.755168199539
7000, 0.09991098940372467, 2190.2636346817017
7037, 0.0998152643442154, 2201.8003046512604
7074, 0.10046721994876862, 2213.3656103610992
7110, 0.09977453202009201, 2224.81085062027
7147, 0.09994886070489883, 2236.3605439662933
7184, 0.1001158356666565, 2247.937453508377
7220, 0.09971576184034348, 2259.326203584671
7257, 0.0997154712677002, 2270.9131445884705
7294, 0.09981860220432281, 2282.523468017578
7331, 0.1005338504910469, 2294.0222039222717
7367, 0.0997912585735321, 2305.465789079666
7404, 0.09982766956090927, 2317.0436398983
7441, 0.099687360227108, 2328.6033256053925
7478, 0.10026182979345322, 2340.1741070747375
7515, 0.10012027621269226, 2351.7045118808746
7552, 0.1003410

13330, 0.0997452363371849, 4173.807742357254
13367, 0.09968326985836029, 4185.413680553436
13404, 0.10002756863832474, 4196.941427707672
13441, 0.09977565705776215, 4208.464646816254
13478, 0.09984669834375381, 4220.000853776932
13515, 0.0996265783905983, 4231.53947019577
13552, 0.09994547069072723, 4243.078602552414
13589, 0.1002294272184372, 4254.650747537613
13625, 0.09980745613574982, 4266.113141536713
13662, 0.1000891625881195, 4277.641272783279
13699, 0.09973807632923126, 4289.17919588089
13736, 0.09970251470804214, 4300.702795267105
13772, 0.10005154460668564, 4312.1550970077515
13809, 0.10009397566318512, 4323.659822702408
13845, 0.099545918405056, 4335.183122634888
13882, 0.09974085539579391, 4346.702016830444
13920, 0.09999562799930573, 4358.350864171982
13956, 0.09977004677057266, 4369.773290395737
13993, 0.1000637486577034, 4381.346323728561
14030, 0.09972649812698364, 4392.915901899338
14066, 0.09988749027252197, 4404.327892780304
14103, 0.09978721290826797, 4415.928493738

19911, 0.09996810555458069, 6238.793831110001
19948, 0.10005050897598267, 6250.376081466675
19985, 0.10021302103996277, 6261.926594495773
20021, 0.09986374527215958, 6273.406919002533
20058, 0.09992146492004395, 6285.042731046677
20094, 0.09969198703765869, 6296.493597269058
20131, 0.09997572749853134, 6308.039298534393
20167, 0.09978260099887848, 6319.514616012573
20204, 0.0999002754688263, 6331.027385234833
20241, 0.09977040439844131, 6342.533606290817
20279, 0.09979493916034698, 6354.178476810455
20316, 0.09975051879882812, 6365.671764373779
20353, 0.0997886136174202, 6377.174568891525
20390, 0.10012666881084442, 6388.6443972587585
20427, 0.1003040224313736, 6400.139702796936
20464, 0.09994981437921524, 6411.669788837433
20502, 0.09997478872537613, 6423.302235841751
20539, 0.09990275651216507, 6434.8153965473175
20576, 0.09984500706195831, 6446.34415769577
20613, 0.10006850957870483, 6457.846480846405
20650, 0.0996432900428772, 6469.39422249794
20688, 0.09966044127941132, 6481.02105

26529, 0.09987016022205353, 8302.570258617401
26566, 0.10007187724113464, 8314.154467105865
26603, 0.09980414062738419, 8325.719614982605
26639, 0.09971415251493454, 8337.210374593735
26676, 0.10004132241010666, 8348.755066633224
26713, 0.09981697797775269, 8360.353742599487
26749, 0.09980075806379318, 8371.779764175415
26786, 0.09973660111427307, 8383.363196849823
26823, 0.09985237568616867, 8394.915796518326
26860, 0.10001236945390701, 8406.444887399673
26897, 0.09978407621383667, 8417.990342378616
26934, 0.10041261464357376, 8429.525468587875
26971, 0.09959523379802704, 8441.088646411896
27008, 0.0998193621635437, 8452.60728931427
27045, 0.09987830370664597, 8464.156847715378
27082, 0.0998561829328537, 8475.673541545868
27119, 0.09973504394292831, 8487.241267681122
27156, 0.1001848503947258, 8498.783476829529
27193, 0.09977249801158905, 8510.327122211456
27230, 0.09974629431962967, 8521.829712867737
27267, 0.09980523586273193, 8533.348836183548
27304, 0.09972810000181198, 8544.87348

...

59163, 0.09971828013658524, 18490.897950172424
59200, 0.10009203106164932, 18502.477006435394
59236, 0.10021506994962692, 18513.954454421997
59272, 0.09989241510629654, 18525.408374786377
59309, 0.10041210800409317, 18536.97042489052
59346, 0.10012015700340271, 18548.492547512054
59383, 0.10038504749536514, 18560.064620018005
59419, 0.09982985258102417, 18571.524722099304
59455, 0.09984970837831497, 18582.991121530533
59492, 0.09961088746786118, 18594.559240579605
59529, 0.0999792292714119, 18606.11084008217
59565, 0.09977898746728897, 18617.600057840347
59602, 0.09993761777877808, 18629.16271996498
59638, 0.09992791712284088, 18640.616297245026
59675, 0.09988684207201004, 18652.17490887642
59712, 0.09996910393238068, 18663.7404859066
59748, 0.10009756684303284, 18675.13022994995
59785, 0.10012006759643555, 18686.71531701088
59821, 0.09973621368408203, 18698.150705337524
59858, 0.09977931529283524, 18709.67331814766
59895, 0.10007468611001968, 18721.24346256256
59931, 0.099841177463531

65634, 0.10028046369552612, 20530.977316379547
65670, 0.1001133844256401, 20542.523607730865
65706, 0.10008393973112106, 20554.067269563675
65743, 0.09978761523962021, 20565.65270781517
65779, 0.09983614832162857, 20577.193830013275
65815, 0.10031779110431671, 20588.750359773636
65851, 0.09993058443069458, 20600.330231189728
65887, 0.10001201927661896, 20611.827372550964
65923, 0.09970209747552872, 20623.31657385826
65959, 0.09969054162502289, 20634.852147102356
65995, 0.09991908818483353, 20646.365307807922
66031, 0.09971616417169571, 20657.933589696884
66067, 0.10028982162475586, 20669.497595071793
66103, 0.0996466651558876, 20681.01771759987
66139, 0.09965086728334427, 20692.449689149857
66175, 0.09984539449214935, 20703.955407857895
66212, 0.09965766221284866, 20715.58423256874
66248, 0.09975803643465042, 20727.09033679962
66284, 0.09965403378009796, 20738.612501621246
66320, 0.09987878054380417, 20750.1176302433
66357, 0.0999951884150505, 20761.709686517715
66393, 0.10005084425210

72046, 0.09987827390432358, 22571.34525680542
72082, 0.10006298869848251, 22582.851546764374
72118, 0.10019982606172562, 22594.343383312225
72155, 0.09972336888313293, 22605.993178129196
72191, 0.09978607296943665, 22617.464977502823
72227, 0.09992343932390213, 22628.945959806442
72263, 0.1001984030008316, 22640.47068786621
72300, 0.10002665221691132, 22652.104147434235
72336, 0.09973696619272232, 22663.59911942482
72372, 0.10003625601530075, 22675.06656718254
72408, 0.09987235069274902, 22686.677461862564
72443, 0.09988336265087128, 22698.16366791725
72479, 0.09985939413309097, 22709.701977968216
72514, 0.10002587735652924, 22721.242440223694
72550, 0.09972814470529556, 22732.758340120316
72586, 0.09985226392745972, 22744.29839491844
72622, 0.09974724054336548, 22755.821497440338
72658, 0.09961600601673126, 22767.350122213364
72694, 0.09987868368625641, 22778.88153553009
72730, 0.09970330446958542, 22790.344861745834
72767, 0.09985186904668808, 22801.936767339706
72803, 0.099911294877

78546, 0.10005085170269012, 24610.514030456543
78583, 0.09999591112136841, 24622.00284767151
78620, 0.09963402152061462, 24633.503506422043
78656, 0.0997428297996521, 24644.907996177673
78693, 0.09974461048841476, 24656.475995779037
78729, 0.09987467527389526, 24667.933186769485
78766, 0.09981128573417664, 24679.51617693901
78803, 0.09992600232362747, 24691.029551029205
78840, 0.09982739388942719, 24702.54929971695
78876, 0.0997680276632309, 24714.00771546364
78913, 0.10035605728626251, 24725.553292274475
78950, 0.09974423795938492, 24737.054723978043
78987, 0.09989064186811447, 24748.592803001404
79023, 0.09994319826364517, 24760.05681157112
79060, 0.09985773265361786, 24771.62811589241
79096, 0.09963633120059967, 24783.084468126297
79133, 0.09983770549297333, 24794.64934206009
79170, 0.1000460609793663, 24806.127792835236
79207, 0.09986680746078491, 24817.6787173748
79244, 0.0998164638876915, 24829.241243600845
79280, 0.0997229740023613, 24840.658477067947
79317, 0.10025287419557571,

85034, 0.10022182017564774, 26646.07652759552
85070, 0.09985148161649704, 26657.595269680023
85107, 0.09998913109302521, 26669.224625349045
85143, 0.1002170592546463, 26680.74376797676
85179, 0.09969998151063919, 26692.25372004509
85215, 0.09963753819465637, 26703.83822607994
85251, 0.10063599795103073, 26715.47511267662
85320, 0.09980543702840805, 26725.596211194992
85406, 0.10027740150690079, 26735.650822639465
85493, 0.09979894757270813, 26745.76329779625
85580, 0.10018953680992126, 26755.809086561203
85667, 0.0998392105102539, 26765.91849589348
85754, 0.09983190149068832, 26775.995974302292
85841, 0.09975258260965347, 26786.008848905563
85928, 0.10016141831874847, 26796.085966348648
86016, 0.09988422691822052, 26806.191323518753
86103, 0.09995560348033905, 26816.247353315353
86190, 0.1001611202955246, 26826.325469732285
86277, 0.09970426559448242, 26836.38400244713
86364, 0.09980706870555878, 26846.44127678871
86451, 0.09959375858306885, 26856.531413555145
86538, 0.0996820256114006

In [15]:
full_h = torch.zeros( (num_subjects, num_nodes, 1), dtype=float_type, device=device )
full_J = torch.zeros( (num_subjects, num_nodes, num_nodes), dtype=float_type, device=device )
start_time = time.time()
last_time = start_time
print_every_seconds = 10
num_epochs = 1000
batch_size = 30
num_batches = num_time_points//batch_size
for node_index in range(num_nodes):
    not_node_indices = torch.arange( data_ts.size(dim=-1), dtype=int_type, device=device ) != node_index
    node_data_ts = data_ts[:,:,node_index].reshape( (num_subjects, num_time_points, 1) )
    not_node_data_ts = data_ts[:,:,not_node_indices].reshape( (num_subjects, num_time_points, num_nodes-1) )
    imnlpln_fn = IsingModelNegativeLogPseudoLikelihoodNode(num_subjects=num_subjects, num_other_nodes=num_nodes-1, dtype=float_type, device=device)
    optimizer = torch.optim.Adam( imnlpln_fn.parameters(), lr=learning_rate )
    for epoch in range(num_epochs):
        time_point_order = torch.randperm(num_time_points, dtype=int_type, device=device)
        node_data_ts_shuffled = node_data_ts[:,time_point_order,:]
        not_node_data_ts_shuffled = not_node_data_ts[:,time_point_order,:]
        for batch in range(num_batches):
            batch_start = batch*batch_size
            batch_end = batch_start+batch_size
            node_data_ts_batch = node_data_ts_shuffled[:,batch_start:batch_end,:]
            not_node_data_ts_batch = not_node_data_ts_shuffled[:,batch_start:batch_end,:]
            optimizer.zero_grad()
            loss = imnlpln_fn(node_data_ts_batch, not_node_data_ts_batch)
            loss.backward()
            optimizer.step()
            current_time = time.time()
        if current_time - last_time >= print_every_seconds:
            loss = imnlpln_fn(node_data_ts, not_node_data_ts)
            print( f'{node_index}, {epoch}, {loss.item()}, {current_time-start_time}' )
            last_time = current_time
    full_h[:,node_index,0] = imnlpln_fn.h_node[:,0,0].detach()
    full_J[:,not_node_indices,node_index] = imnlpln_fn.J_not_node_vs_node[:,:,0].detach()
    loss = imnlpln_fn(node_data_ts, not_node_data_ts)
    print( f'{node_index}, {epoch}, {loss.item()}, {current_time-start_time}' )

0, 84, 0.18742579221725464, 10.05046558380127
0, 168, 0.11251689493656158, 20.123863220214844
0, 249, 0.10251996666193008, 30.1311194896698
0, 330, 0.10059879720211029, 40.2449996471405
0, 411, 0.10051126778125763, 50.317997455596924
0, 492, 0.09985849261283875, 60.36608576774597
0, 573, 0.09986401349306107, 70.48495078086853
0, 653, 0.09990429878234863, 80.49613571166992
0, 735, 0.10030373185873032, 90.54516553878784
0, 816, 0.10009361803531647, 100.59820890426636
0, 896, 0.09989438951015472, 110.64529800415039
0, 975, 0.10042440891265869, 120.70230269432068
0, 999, 0.10026080161333084, 123.7137360572815
1, 56, 0.37541112303733826, 130.72097277641296
1, 139, 0.17331384122371674, 140.7535526752472
1, 223, 0.15583309531211853, 150.86004757881165
1, 306, 0.15389284491539001, 160.91813206672668
1, 390, 0.1535351276397705, 171.02101373672485
1, 473, 0.1530158966779709, 181.05473971366882
1, 558, 0.15388739109039307, 191.14246225357056
1, 644, 0.1528390347957611, 201.19939184188843
1, 730, 

13, 830, 0.20425282418727875, 1628.982943058014
13, 915, 0.20332221686840057, 1639.0734238624573
13, 998, 0.20380111038684845, 1649.118041753769
13, 999, 0.20356863737106323, 1649.2407133579254
14, 82, 0.23701153695583344, 1659.1740520000458
14, 167, 0.15130756795406342, 1669.2491247653961
14, 253, 0.14050833880901337, 1679.3463170528412
14, 339, 0.13942953944206238, 1689.415366411209
14, 425, 0.13875429332256317, 1699.4215457439423
14, 509, 0.13946214318275452, 1709.4961905479431
14, 595, 0.13886551558971405, 1719.6130592823029
14, 682, 0.1387450098991394, 1729.7073352336884
14, 769, 0.13899710774421692, 1739.753398180008
14, 855, 0.13889871537685394, 1749.797308921814
14, 941, 0.1386284977197647, 1759.8016376495361
14, 999, 0.1397886425256729, 1766.4826939105988
15, 28, 0.7299869656562805, 1769.8116059303284
15, 115, 0.16909393668174744, 1779.8220570087433
15, 201, 0.13347947597503662, 1789.857265472412
15, 289, 0.12878549098968506, 1799.9219422340393
15, 372, 0.12815409898757935, 18

27, 318, 0.16866719722747803, 3207.942722558975
27, 405, 0.16813240945339203, 3217.9818394184113
27, 492, 0.16793164610862732, 3228.0000121593475
27, 580, 0.16807502508163452, 3238.0861835479736
27, 667, 0.16812904179096222, 3248.1249573230743
27, 755, 0.16792291402816772, 3258.1968324184418
27, 842, 0.16823211312294006, 3268.2641382217407
27, 930, 0.1677847057580948, 3278.3708810806274
27, 999, 0.16794505715370178, 3286.3064274787903
28, 17, 1.2929950952529907, 3288.4178919792175
28, 102, 0.21888746321201324, 3298.4975810050964
28, 183, 0.1668596714735031, 3308.553190946579
28, 266, 0.16154804825782776, 3318.6373109817505
28, 350, 0.159810408949852, 3328.6822023391724
28, 435, 0.1603490561246872, 3338.732778072357
28, 521, 0.160415381193161, 3348.8323130607605
28, 607, 0.16007451713085175, 3358.940172433853
28, 695, 0.1604161113500595, 3369.0458760261536
28, 782, 0.16001074016094208, 3379.106364250183
28, 869, 0.16012994945049286, 3389.1276721954346
28, 957, 0.16044993698596954, 3399.

40, 686, 0.1958436667919159, 4810.9863748550415
40, 714, 0.19635017216205597, 4822.407322883606
40, 742, 0.1962064951658249, 4833.439743757248
40, 783, 0.1963924914598465, 4843.474385976791
40, 868, 0.19609247148036957, 4853.546387195587
40, 952, 0.1957165151834488, 4863.582481622696
40, 999, 0.1961020827293396, 4869.73099899292
41, 28, 0.714483380317688, 4873.631049156189
41, 98, 0.18925489485263824, 4883.740953445435
41, 183, 0.13485866785049438, 4893.776089668274
41, 269, 0.1283894181251526, 4903.869033098221
41, 355, 0.12686479091644287, 4913.908118247986
41, 439, 0.12702931463718414, 4924.005049943924
41, 526, 0.12670905888080597, 4934.083079814911
41, 613, 0.12638388574123383, 4944.130146980286
41, 700, 0.12677599489688873, 4954.20477437973
41, 787, 0.1263338178396225, 4964.263002872467
41, 862, 0.12641693651676178, 4974.606367588043
41, 895, 0.12663553655147552, 4984.678503036499
41, 919, 0.12666228413581848, 4994.756343126297
41, 944, 0.12658263742923737, 5005.025149345398
41, 

45, 999, 0.12444842606782913, 6639.620196819305
46, 5, 2.2130258083343506, 6642.302144765854
46, 39, 0.5181844830513, 6652.394880771637
46, 70, 0.2796882688999176, 6664.085614681244
46, 104, 0.19520220160484314, 6674.1397931575775
46, 133, 0.16587242484092712, 6685.772875070572
46, 218, 0.14316920936107635, 6695.776979446411
46, 304, 0.14052385091781616, 6705.786707162857
46, 392, 0.14028328657150269, 6715.888485431671
46, 478, 0.13999898731708527, 6725.914854288101
46, 564, 0.14018145203590393, 6735.940411090851
46, 651, 0.14005790650844574, 6746.01427936554
46, 737, 0.13984012603759766, 6756.040820360184
46, 821, 0.14010269939899445, 6766.0686984062195
46, 906, 0.13980013132095337, 6776.163638114929
46, 993, 0.13978517055511475, 6786.20877456665
46, 999, 0.14039811491966248, 6786.900486469269
47, 77, 0.2616654932498932, 6796.258410215378
47, 161, 0.1519128531217575, 6806.270471096039
47, 246, 0.13911524415016174, 6816.358036994934
47, 332, 0.13678854703903198, 6826.37019944191
47, 41

...

335, 257, 0.15678563714027405, 82011.34722661972
335, 293, 0.1556307077407837, 82022.57831406593
335, 329, 0.15563690662384033, 82033.82123923302
335, 365, 0.15549501776695251, 82045.0345146656
335, 402, 0.1552576869726181, 82056.31764221191
335, 438, 0.1553933024406433, 82067.53509187698
335, 474, 0.15497322380542755, 82078.7535982132
335, 511, 0.15492740273475647, 82090.00088357925
335, 547, 0.1550009697675705, 82101.23037266731
335, 583, 0.1550663560628891, 82112.47779250145
335, 620, 0.155497208237648, 82123.7974729538
335, 656, 0.15535826981067657, 82135.02919411659
335, 692, 0.15493589639663696, 82146.31008172035
335, 727, 0.15483324229717255, 82157.42432260513
335, 764, 0.1550486832857132, 82168.73899817467
335, 800, 0.1549212485551834, 82179.96693253517
335, 836, 0.15467412769794464, 82191.21985411644
335, 872, 0.1547045111656189, 82202.39596366882
335, 908, 0.1548214852809906, 82213.71706080437
335, 942, 0.1553686559200287, 82224.94747829437
335, 977, 0.15552951395511627, 8223

341, 150, 0.2222786396741867, 83855.15911221504
341, 186, 0.21166624128818512, 83866.41511797905
341, 222, 0.20806123316287994, 83877.64602804184
341, 258, 0.20721225440502167, 83888.91881871223
341, 294, 0.20587614178657532, 83900.13726949692
341, 330, 0.20559804141521454, 83911.37567925453
341, 366, 0.20639672875404358, 83922.59632086754
341, 402, 0.20552290976047516, 83933.8960442543
341, 438, 0.20608927309513092, 83945.14888644218
341, 473, 0.2060856968164444, 83956.32857275009
341, 509, 0.20700955390930176, 83967.65377211571
341, 545, 0.20621484518051147, 83978.86747789383
341, 581, 0.20606699585914612, 83990.11102557182
341, 617, 0.20607724785804749, 84001.38282585144
341, 653, 0.2057667225599289, 84012.65862250328
341, 689, 0.2062215954065323, 84023.86076378822
341, 725, 0.20649385452270508, 84035.10367822647
341, 761, 0.20590399205684662, 84046.33858156204
341, 798, 0.20576342940330505, 84057.60396337509
341, 834, 0.20576287806034088, 84068.83286857605
341, 869, 0.2053684294223

347, 41, 0.6883749961853027, 85699.7048933506
347, 76, 0.35261863470077515, 85710.89749765396
347, 111, 0.26308199763298035, 85722.13278770447
347, 146, 0.2376738041639328, 85733.31364679337
347, 182, 0.2300267368555069, 85744.64440274239
347, 217, 0.22768180072307587, 85755.81048512459
347, 253, 0.22742648422718048, 85767.12077474594
347, 288, 0.22693495452404022, 85778.35012245178
347, 323, 0.2268039882183075, 85789.54119229317
347, 359, 0.22703397274017334, 85800.8515996933
347, 395, 0.2271251529455185, 85812.08549547195
347, 430, 0.2266598343849182, 85823.24273991585
347, 467, 0.22666430473327637, 85834.59859633446
347, 503, 0.22718070447444916, 85845.82703375816
347, 539, 0.22737552225589752, 85857.07471561432
347, 575, 0.22684234380722046, 85868.33758544922
347, 611, 0.22665153443813324, 85879.53270506859
347, 647, 0.2277543544769287, 85890.79158043861
347, 683, 0.22679294645786285, 85902.08131098747
347, 719, 0.226889967918396, 85913.2588801384
347, 756, 0.22824664413928986, 859

352, 945, 0.21185176074504852, 87556.13300395012
352, 981, 0.21153832972049713, 87567.3407740593
352, 999, 0.2114231139421463, 87571.14163923264
353, 18, 1.2058488130569458, 87578.61513876915
353, 54, 0.4002726972103119, 87589.76124310493
353, 90, 0.23205223679542542, 87600.97971248627
353, 127, 0.1853616088628769, 87612.28242969513
353, 164, 0.17051999270915985, 87623.55821633339
353, 199, 0.1655949205160141, 87634.74753785133
353, 234, 0.16336213052272797, 87645.97957658768
353, 269, 0.16283029317855835, 87657.21770048141
353, 305, 0.1623537540435791, 87668.41952300072
353, 341, 0.1619955450296402, 87679.75312304497
353, 377, 0.1622554361820221, 87690.97355937958
353, 412, 0.16230027377605438, 87702.17016768456
353, 447, 0.1618214100599289, 87713.37932944298
353, 483, 0.1619763821363449, 87724.65213680267
353, 519, 0.16199010610580444, 87735.9139573574
353, 554, 0.16182677447795868, 87747.09699869156
353, 590, 0.16251200437545776, 87758.37107348442
353, 626, 0.16223202645778656, 8776

358, 853, 0.1712433099746704, 89400.1021823883
358, 889, 0.17133253812789917, 89411.33011841774
358, 926, 0.1723686158657074, 89422.61789679527
358, 963, 0.17150527238845825, 89433.90397047997
358, 999, 0.1713697910308838, 89445.1011171341
358, 999, 0.1713697910308838, 89445.1011171341
359, 35, 0.9088044166564941, 89456.34165024757
359, 72, 0.42435309290885925, 89467.64686107635
359, 108, 0.3014986515045166, 89478.83089900017
359, 144, 0.2650688588619232, 89490.1396291256
359, 179, 0.25463148951530457, 89501.26890039444
359, 216, 0.25166967511177063, 89512.58756136894
359, 252, 0.25038883090019226, 89523.80955529213
359, 288, 0.25051063299179077, 89535.0169634819
359, 325, 0.25012513995170593, 89546.3260602951
359, 361, 0.24929527938365936, 89557.52963089943
359, 397, 0.2503279447555542, 89568.77803587914
359, 433, 0.24941039085388184, 89579.92580652237
359, 469, 0.2498399168252945, 89591.2205491066
359, 504, 0.24979032576084137, 89602.35675573349
359, 540, 0.24989908933639526, 89613.5

In [90]:
def run_ising_model_sim(h:torch.Tensor, J:torch.Tensor, num_time_points:int):
    num_subjects, num_nodes, _ = h.size()
    print(num_subjects, num_nodes)
    sim_ts = torch.zeros( (num_subjects, num_time_points, num_nodes), dtype=float_type, device=device )
    num_flips = 0
    # state = 2.0*torch.randint( low=0, high=1, size=(num_subjects, 1, num_nodes), device=device ) - 1.0
    # print( state.size() )
    # all_indices = torch.arange( data_ts.size(dim=-1), dtype=int_type, device=device )
    for t in range(num_time_points):
        # node_order = torch.randperm(n=num_nodes, dtype=int_type, device=device)
        for i in range(num_nodes):
            print(f'{i}', end="")
            # node_index = node_order[i]
            # not_node_indices = all_indices != node_index
            # node_state = state[:,:,node_index].reshape( (num_subjects, 1, 1) )
            # not_node_state = state[:,:,not_node_indices].reshape( (num_subjects, 1, num_nodes-1) )
            # h_node = h[:,node_index].reshape( (num_subjects, 1, 1) )
            # J_not_node_vs_node = J[:,not_node_indices,node_index].reshape( (num_subjects, num_nodes-1, 1) )
            # deltaH = 2 * node_state * ( h_node + torch.matmul(not_node_state, J_not_node_vs_node) )
            # print( deltaH.size() )
            # prob_accept = torch.clamp( torch.exp(-beta*deltaH), min=0.0, max=1.0 )
            # flip = torch.bernoulli(prob_accept)
            # print( flip.size() )
            # print( state.size() )
            # state[:,0,node_index] *= ( 1.0 - 2.0*flip[:,0,0] )
            # num_flips += torch.count_nonzero(flip)
        # sim_ts[:,t,:] = state
        # for node_index in range(num_nodes):
        #     if s[0,0,node_index] > 0:
        #         print('+'),
        #     else:
        #         print('-'),
        print(f'|{t}')
    return sim_ts, num_flips

In [92]:
sim_ts, num_flips = run_ising_model_sim(h=full_h, J=full_J, num_time_points=num_time_points)

10 360


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
torch.save(full_h, 'pseuodlikelihood_test_h_v2.pt')

In [27]:
torch.save(full_J, 'pseuodlikelihood_test_J_v1.pt')

In [28]:
print(full_h.isnan().count_nonzero())

tensor(4, device='cuda:0')


In [29]:
print(full_J.isnan().count_nonzero())

tensor(1436, device='cuda:0')
